In [79]:
import os 
os.chdir('../../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.baseline_functions as base

# restore saved variables
%store -r summary_drug_two_year

In [2]:
### train data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_data.csv")
x = data.loc[:,:'five_year']
y = data['recid_violent6'].values

In [ ]:
#### Logistic
c = [1e-5]
logistic_summary = base.Logistic(X=x,
                                 Y=y, 
                                 C=c,
                                 seed=816)

#### Lasso
c = [0.01]
lasso_summary = base.Lasso(X=x,
                           Y=y,
                           C=c,
                           seed=816)

#### LinearSVM
c = [5e-5]
svm_summary = base.LinearSVM(X=x,
                             Y=y,
                             C=c,
                             seed=816)

#### CART
depth = [1]
impurity = [0.004]
split = [2,3,4]
cart_summary = base.CART(X=x,
                         Y=y,
                         depth=depth,
                         split=split,
                         impurity=impurity, 816)

#### Random Forest
n_estimators =  [20,40,60,80]
depth = [1]
impurity = [0.003]
rf_summary = base.RF(X=x,
                     Y=y, 
                     depth=depth, 
                     estimators=n_estimators, 
                     impurity=impurity,
                     seed=816)

#### XGBoost
learning_rate = [0.05]
depth = [1]
n_estimators = [20]
gamma = [2,4,6]
child_weight = [2,4,6]
subsample = [0.3, 0.5]
xgb_summary = base.XGB(X=x,
                       Y=y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       gamma=gamma, 
                       child_weight=child_weight, 
                       subsample=subsample, 
                       seed=816)

#### save results
summary_drug_two_year = {"Logistic": logistic_summary,
                         "Lasso": lasso_summary,
                         "LinearSVM": svm_summary,
                         "CART": cart_summary,
                         "RF": rf_summary,
                         "XGBoost": xgb_summary}
%store summary_drug_two_year

In [ ]:
results = [[model_name, np.mean(model_summary["holdout_test_auc"]), np.mean(model_summary["auc_diffs"])] \
           for model_name, model_summary in summary_drug_two_year.items()]
results 

In [ ]:
params = [[model_name, model_summary['best_param']] for model_name, model_summary in summary_drug_two_year.items()]
params

In [168]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\broward\\broward models\\model results\\Baselines\\Six Month\\"
results = [["", "Logistic", "STD", "Lasso", "STD", "Linear SVM", "STD", "CART", "STD", "Random Forest", "STD", "XGBoost", "STD"],
           ["Violent", np.str((round(np.mean(logistic_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(logistic_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(lasso_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(lasso_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(svm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(svm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(cart_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(rf_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(rf_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(xgb_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(xgb_summary['holdout_test_auc']), 3)) + ")"]]
with open(path + 'Six Month Baseline Summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)